In [10]:
# 데이터 가져오기
from google.colab import drive
drive.mount('/content/gdrive/')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
# One_Class Support Vector Machine , AutoEncoder, Isololation Forest 

path = "/content/gdrive/Shareddrives/DACON_fds/source/fds/"
# 샘플 결과 저장 
submission_path = "/content/gdrive/Shareddrives/DACON_fds/source/submission/"

train = pd.read_csv(path + "train.csv")
test = pd.read_csv(path +"test.csv")
val = pd.read_csv(path +"val.csv")
sample = pd.read_csv(path +"sample_submission.csv")


Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics.pairwise import cosine_similarity

train = pd.read_csv("open/train.csv")
val = pd.read_csv("open/val.csv")

train.drop(columns = {"ID"},inplace=True)
val.drop(columns = {"ID"},inplace=True)

test = val[val.columns[:-1]]

nFeature=train.shape[1]
inputX = Input(batch_shape=(None,nFeature))

Xencode = Dense(12, activation='relu')(inputX)
Xdecode = Dense(nFeature, activation='linear')(Xencode)

model = Model(inputX, Xdecode)
model.compile(loss='mse', optimizer=Adam(lr=0.001))

h = model.fit(train,train,epochs=300, batch_size=1000, shuffle=True)
yhat=model.predict(test)

dist_list=[]
for i in range(len(yhat)):
    dist = cosine_similarity([test.iloc[0].values],[yhat[0]])
    dist_list.append(dist)

dist_list2 = dist_list.copy()
maxN=492
max_dict={}

for i in range(maxN):
    max_index = dist_list2.index(max(dist_list2))
    max_value = dist_list2.pop(max_index)
    dist_list2.insert(max_index,0)
    max_dict[max_index]=max_value

actual =set(val[val['Class']==1].index)
predicted = set(max_dict.keys())

print('실제로 비정상인 데이터 수:',len(actual))
print('실제 비정상인데 비정상으로 예측한 데이터 수:',len(actual.intersection(predicted)))
print('precision={}'.format(len(actual.intersection(predicted))/len(actual)))


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.losses import MeanSquaredLogarithmicError

train = pd.read_csv("open/train.csv")
val = pd.read_csv("open/val.csv")

train.drop(columns = {"ID"},inplace=True)
val.drop(columns = {"ID"},inplace=True)

tf.enable_eager_execution()



class AutoEncoder(Model):
    def __init__(self, output_units, code_size=8):
        super().__init__()
        self.encoder = Sequential([
          Dense(64, activation='relu'),
          Dropout(0.1),
          Dense(32, activation='relu'),
          Dropout(0.1),
          Dense(16, activation='relu'),
          Dropout(0.1),
          Dense(code_size, activation='relu')
        ])
        self.decoder = Sequential([
          Dense(16, activation='relu'),
          Dropout(0.1),
          Dense(32, activation='relu'), ㅇ
          Dropout(0.1),
          Dense(64, activation='relu'),
          Dropout(0.1),
          Dense(output_units, activation='sigmoid')
        ])

    def call(self, inputs):
        encoded = self.encoder(inputs)
        decoded = self.decoder(encoded)
        return decoded
  

model = AutoEncoder(output_units=train.shape[1])
model.compile(loss='msle', metrics=['mse'], optimizer='adam')

std_scaler = StandardScaler()
std_scaler.fit(train)
x_train = std_scaler.transform(train)
x_test = std_scaler.transform(val[val.columns[:-1]])

history = model.fit(
    x_train,
    x_train,
    epochs=20,
    batch_size=512,
    validation_data=(x_test, x_test)
)

def find_threshold(model, x_train_scaled):
    reconstructions = model.predict(x_train_scaled)
      # provides losses of individual instances
    reconstruction_errors = tf.keras.losses.msle(reconstructions, x_train_scaled)
      # threshold for anomaly scores
    threshold = np.mean(reconstruction_errors.numpy()) \
+ np.std(reconstruction_errors.numpy())
    return threshold

def get_predictions(model, x_test_scaled, threshold):
    predictions = model.predict(x_test_scaled)
      # provides losses of individual instances
    errors = tf.keras.losses.msle(predictions, x_test_scaled)
      # 0 = anomaly, 1 = normal
    anomaly_mask = pd.Series(errors) > threshold
    preds = anomaly_mask.map(lambda x: 1.0 if x == True else 0.0)
    
    return preds

threshold = find_threshold(model, x_train)
print(f"Threshold: {threshold}")

# Threshold: 0.01001314025746261
predictions = get_predictions(model, x_test, threshold)

predictions.value_counts()

from sklearn.metrics import classification_report
print(classification_report(val['Class'].values,predictions))

predictions = get_predictions(model, x_test, threshold)

In [ ]:
test = pd.read_csv("open/test.csv")

test.drop(columns = {"ID"},inplace=True)
scale_test = std_scaler.transform(test)
test['Class'] = get_predictions(model, scale_test, threshold)

sample = pd.read_csv("open/sample_submission.csv")
sample['Class'] = test['Class']
sample.to_csv("open/sample_submission.csv",index=False)